In [2]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.0/930.0 kB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 94.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 112.3 MB/s eta 0:00:00


In [8]:
import torch
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import pennylane as qml
from pennylane import numpy as np
from datetime import datetime

In [9]:
# Check if CUDA is available
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [11]:
# Define the transformation to resize the images to 224x224
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resizing to 224x224
    transforms.ToTensor()  # Convert the image to PyTorch tensor
])

In [12]:
# Download and load CIFAR-10 dataset with the transformation
train_data = datasets.CIFAR10(
    root='data',
    train=True,
    transform=transform,  # Apply the transform
    download=True
)

test_data = datasets.CIFAR10(
    root='data',
    train=False,
    transform=transform  # Apply the transform
)

100%|██████████| 170M/170M [00:05<00:00, 30.3MB/s]


Extracting data/cifar-10-python.tar.gz to data


In [13]:
# Define the quantum circuit using PennyLane
n_qubits = 5
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnode(inputs, weights):
    qml.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]

# Define the QLayer
n_layers = 3
weight_shapes = {"weights": (n_layers, n_qubits)}

In [14]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 16, 5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm2d(16)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm2d(32)

        # Fully connected layers before quantum layers
        self.fc1 = nn.Linear(32 * 56 * 56, 5)  # Output size must match quantum layer input size (5)

        # Quantum layers
        self.qlayer = qml.qnn.TorchLayer(qnode, weight_shapes)

        # Final fully connected layers
        self.fc2 = nn.Linear(5, 10)  # Output size is 10 (for CIFAR-10 classes)

    def forward(self, x):
        # Apply convolutional layers
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))

        # Flatten the tensor
        x = x.view(x.size(0), -1)  # This handles the flattening dynamically

        # Fully connected layer (output size is now 5 to match quantum layer)
        x = F.relu(self.fc1(x))

        # Quantum layer
        x = self.qlayer(x)

        # Final fully connected layer
        x = self.fc2(x)
        return x

In [15]:
# Initialize the model
cnn = Net().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for classification
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)  # Adam optimizer for better convergence

# Split the dataset into training and validation sets
train_size = int(0.8 * len(train_data))
val_size = len(train_data) - train_size
train_set, val_set = torch.utils.data.random_split(train_data, [train_size, val_size])

In [16]:
# Create DataLoader for the training and validation sets
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
val_loader = DataLoader(val_set, batch_size=64, shuffle=False)

In [18]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    start_time = datetime.now()
    print(f"Epoch {epoch+1}/{num_epochs}, {start_time}")

    running_loss = 0.0
    cnn.train()  # Set the model to training mode
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  # Zero the parameter gradients
        outputs = cnn(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute the loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update the model parameters

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")

    # Validation step (optional)
    cnn.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  # Disable gradient calculation for validation
        for data in val_loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = cnn(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = 100 * correct / total
    print(f"Validation Accuracy: {val_accuracy:.2f}%")

print('Finished Training')

Epoch 1/10, 2025-03-08 07:33:17.190212
Epoch 1, Loss: 2.3072
Validation Accuracy: 10.11%
Epoch 2/10, 2025-03-08 07:35:48.731250
Epoch 2, Loss: 2.3030
Validation Accuracy: 9.74%
Epoch 3/10, 2025-03-08 07:38:22.867166
Epoch 3, Loss: 2.3027
Validation Accuracy: 10.06%
Epoch 4/10, 2025-03-08 07:40:57.248240
Epoch 4, Loss: 2.3026
Validation Accuracy: 10.64%
Epoch 5/10, 2025-03-08 07:43:31.382648
Epoch 5, Loss: 2.3025
Validation Accuracy: 9.83%
Epoch 6/10, 2025-03-08 07:46:05.285154
Epoch 6, Loss: 2.3029
Validation Accuracy: 8.51%
Epoch 7/10, 2025-03-08 07:48:36.553464
Epoch 7, Loss: 2.3026
Validation Accuracy: 9.72%
Epoch 8/10, 2025-03-08 07:51:09.298744
Epoch 8, Loss: 2.3028
Validation Accuracy: 10.03%
Epoch 9/10, 2025-03-08 07:53:41.977507
Epoch 9, Loss: 2.3029
Validation Accuracy: 9.74%
Epoch 10/10, 2025-03-08 07:56:13.277242
Epoch 10, Loss: 2.3029
Validation Accuracy: 9.68%
Finished Training
